In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
from google.colab import drive

#  Monter Google Drive
drive.mount('/content/drive')

#  Chemins
drive_base_path = "/content/drive/MyDrive/Colab Notebooks/fahim/final"
local_base_path = "/content/modeles_importes"
os.makedirs(local_base_path, exist_ok=True)

#  1. Modèles à copier (images + textuels)
model_folders = {
    "modeles_image_v2": ["efficientv2L_best.pt", "convnextv2_best.pt", "coatnet2_best.pt","Maxvit_best.pt", "gcvit_best.pt"],
    "camembert2_model": ["config.json", "model.safetensors"],
    "flaubert2_model": ["config.json", "model.safetensors"],
}

# 📥Copie des modèles
for folder, files in model_folders.items():
    src_dir = os.path.join(drive_base_path, folder)
    is_text_model = "camembert" in folder or "flaubert" in folder
    dst_subfolder = os.path.join(local_base_path, os.path.basename(folder)) if is_text_model else local_base_path
    os.makedirs(dst_subfolder, exist_ok=True)

    for filename in files:
        src_file = os.path.join(src_dir, filename)
        dst_file = os.path.join(dst_subfolder, filename)
        if os.path.exists(src_file):
            shutil.copy(src_file, dst_file)
            print(f"✅ Copié : {filename} → {dst_file}")
        else:
            print(f"❌ Introuvable : {src_file}")

#  2. Logits à copier (texte + image + labels)
logits_files = [
    "efficientv2L_logits.pt",
    "convnextv2_logits.pt",
    "coatnet2_logits.pt",
    "Maxvit_logits.pt",
    "gcvit_logits.pt",
    "camembert2_logits.pt",
    "flaubert2_logits.pt",
    "true_labels_final.pt",
    "booster_logits.pt"
]

logits_drive_path = drive_base_path

for filename in logits_files:
    src_file = os.path.join(logits_drive_path, filename)
    dst_file = os.path.join(local_base_path, filename)
    if os.path.exists(src_file):
        shutil.copy(src_file, dst_file)
        print(f"✅ Logits copié : {filename}")
    else:
        print(f"❌ Logits manquant : {src_file}")
# 3. Copier le fichier de mapping labels
mapping_filename = "label_mapping_final.json"
mapping_src = os.path.join(drive_base_path, mapping_filename)
mapping_dst = os.path.join(local_base_path, mapping_filename)

if os.path.exists(mapping_src):
    shutil.copy(mapping_src, mapping_dst)
    print(f"✅ Mapping copié : {mapping_filename}")
else:
    print(f"❌ Mapping manquant : {mapping_src}")
# 4. Copier le fichier label encoder
label_encoder_filename = "label_encoder_final.pkl"
label_encoder_src = os.path.join(drive_base_path, label_encoder_filename)
label_encoder_dst = os.path.join(local_base_path, label_encoder_filename)

if os.path.exists(label_encoder_src):
    shutil.copy(label_encoder_src, label_encoder_dst)
    print(f"✅ Label Encoder copié : {label_encoder_filename}")
else:
    print(f"❌ Label Encoder manquant : {label_encoder_src}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Copié : efficientv2L_best.pt → /content/modeles_importes/efficientv2L_best.pt
✅ Copié : convnextv2_best.pt → /content/modeles_importes/convnextv2_best.pt
✅ Copié : coatnet2_best.pt → /content/modeles_importes/coatnet2_best.pt
✅ Copié : Maxvit_best.pt → /content/modeles_importes/Maxvit_best.pt
✅ Copié : gcvit_best.pt → /content/modeles_importes/gcvit_best.pt
✅ Copié : config.json → /content/modeles_importes/camembert2_model/config.json
✅ Copié : model.safetensors → /content/modeles_importes/camembert2_model/model.safetensors
✅ Copié : config.json → /content/modeles_importes/flaubert2_model/config.json
✅ Copié : model.safetensors → /content/modeles_importes/flaubert2_model/model.safetensors
✅ Logits copié : efficientv2L_logits.pt
✅ Logits copié : convnextv2_logits.pt
✅ Logits copié : coatnet2_logits.pt
✅ Logits copié : Maxvit_logits.pt
✅ Logits copié : gcvit_

In [ ]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 23.6 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import torch
import json
from scipy.special import softmax
from sklearn.metrics import f1_score
import optuna
from itertools import combinations

# --- Config ---
BASE = "/content/drive/MyDrive/Colab Notebooks/fahim/final"
PATH_LOGITS = "/content/modeles_importes"
VAL_INDICES_PATH = os.path.join(BASE, "val_indices.json")
LABELS_PATH = os.path.join(PATH_LOGITS, "true_labels_final.pt")

# --- Modèles fixes ---
fixed_models = ["booster_logits.pt", "camembert2_logits.pt", "flaubert2_logits.pt"]

# --- Modèles optionnels ---
optional_models = [
    "convnextv2_logits.pt",
    "Maxvit_logits.pt",
    "gcvit_logits.pt",
    "coatnet2_logits.pt",
    "efficientv2L_logits.pt",
]

# --- Charger validation indices et labels ---
with open(VAL_INDICES_PATH, "r") as f:
    val_idx = np.array(json.load(f))
labels_full = torch.load(LABELS_PATH).numpy()
y_val = labels_full[val_idx]

# --- Fonction softmax + chargement logits ---
def load_logits(filenames):
    logits_list = []
    for fname in filenames:
        logits = torch.load(os.path.join(PATH_LOGITS, fname)).cpu().numpy()
        if logits.ndim == 2 and (np.abs(np.sum(logits, axis=1) - 1) > 1e-4).any():
            logits = softmax(logits, axis=1)
        logits_val = logits[val_idx]
        logits_list.append(logits_val)
    return logits_list

# --- Fonction fusion simple (poids globaux) ---
def weighted_fusion(logits_list, weights):
    stacked = np.stack(logits_list, axis=0)  # (n_models, n_samples, n_classes)
    weighted = np.tensordot(weights, stacked, axes=(0, 0))  # (n_samples, n_classes)
    return weighted

# --- Fonction objective Optuna ---
def make_objective(logits_list, y_true):
    n_models = len(logits_list)
    def objective(trial):
        weights = []
        for i in range(n_models):
            w = trial.suggest_float(f"w_{i}", 0.0, 1.0)
            weights.append(w)
        s = sum(weights)
        if s == 0:
            return 1.0
        weights = [w / s for w in weights]
        fused_logits = weighted_fusion(logits_list, weights)
        preds = np.argmax(fused_logits, axis=1)
        f1 = f1_score(y_true, preds, average="weighted")
        print(f"Trial {trial.number}: F1={f1:.4f}")
        return 1 - f1
    return objective

# --- Tester toutes les combinaisons ---
best_score = 0
best_combo = None
best_weights = None

# Toutes tailles de combinaisons possibles sur optional_models
for r in range(len(optional_models) + 1):
    for combo in combinations(optional_models, r):
        models_to_use = fixed_models + list(combo)
        print(f"\nTest combo: {models_to_use}")
        logits_list = load_logits(models_to_use)
        objective = make_objective(logits_list, y_val)
        study = optuna.create_study(direction="minimize")
        study.optimize(objective, n_trials=300, n_jobs=3)  # 30 trials par combo pour gagner du temps
        current_f1 = 1 - study.best_value
        print(f"Combo {models_to_use} - Best F1: {current_f1:.4f}")
        if current_f1 > best_score:
            best_score = current_f1
            best_combo = models_to_use
            n_models = len(logits_list)
            best_weights = np.zeros(n_models)
            for i in range(n_models):
                best_weights[i] = study.best_params[f"w_{i}"]

print("\n--- Résultat final ---")
print(f"Meilleure combinaison : {best_combo}")
print(f"Meilleur F1-weighted : {best_score:.4f}")
print(f"Poids optimaux : {best_weights}")


In [ ]:
import os
import numpy as np
import torch
import json
from scipy.special import softmax
from sklearn.metrics import f1_score, classification_report
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import optuna
import joblib

# --- Config chemins ---
BASE = "/content/drive/MyDrive/Colab Notebooks/fahim/final"
PATH_LOGITS = "/content/modeles_importes"
VAL_INDICES_PATH = os.path.join(BASE, "val_indices.json")
LABEL_MAPPING_PATH = os.path.join(BASE, "label_mapping_final.json")
LABELS_PATH = os.path.join(PATH_LOGITS, "true_labels_final.pt")

OUTPUT_DIR = os.path.join(BASE, "meta_multimode_finalV2")
os.makedirs(OUTPUT_DIR, exist_ok=True)

best_models = [
    "booster_logits.pt",
    "camembert2_logits.pt",
    "flaubert2_logits.pt",
    "efficientv2L_logits.pt"
]

fixed_weights = np.array([0.76947969, 0.03915019, 0.13169743, 0.18129248])

# --- Chargement indices, labels ---
with open(VAL_INDICES_PATH, "r") as f:
    val_idx = np.array(json.load(f))
with open(LABEL_MAPPING_PATH, "r") as f:
    label_mapping = json.load(f)
class_names = [label_mapping[str(i)]["label_name"] for i in range(len(label_mapping))]

labels_full = torch.load(LABELS_PATH).numpy()
n_samples = labels_full.shape[0]
all_indices = np.arange(n_samples)
train_idx = np.setdiff1d(all_indices, val_idx)

y_train = labels_full[train_idx]
y_val = labels_full[val_idx]

def load_logits(fnames, indices):
    logits_list = []
    for fname in fnames:
        logits = torch.load(os.path.join(PATH_LOGITS, fname)).cpu().numpy()
        if logits.ndim == 2 and (np.abs(np.sum(logits, axis=1) - 1) > 1e-4).any():
            logits = softmax(logits, axis=1)
        logits = logits[indices]
        logits_list.append(logits)
    return logits_list

logits_train_list = load_logits(best_models, train_idx)
logits_val_list = load_logits(best_models, val_idx)

def weighted_fusion(logits_list, weights):
    stacked = np.stack(logits_list, axis=0)
    weighted = np.tensordot(weights, stacked, axes=(0, 0))
    return weighted

# Fusion objective pour Optuna
def objective_fusion(trial, probas_list, y_true):
    n_weights = len(probas_list)
    weights = []
    for i in range(n_weights):
        weights.append(trial.suggest_float(f'w{i}', 0.0, 1.0))
    weights = np.array(weights)
    if weights.sum() == 0:
        return 1.0
    weights /= weights.sum()
    fusion = weighted_fusion(probas_list, weights)
    preds = np.argmax(fusion, axis=1)
    return 1 - f1_score(y_true, preds, average='weighted')

def save_model(model, path):
    if isinstance(model, nn.Module):
        torch.save(model.state_dict(), path)
    else:
        joblib.dump(model, path)

def load_model(path, model_type, model_class=None, *args, **kwargs):
    if model_type == 'mlp':
        model = model_class(*args, **kwargs)
        model.load_state_dict(torch.load(path))
        model.eval()
        return model
    else:
        return joblib.load(path)

# Logistic Regression
def train_logreg_optuna(trial, X_train, y_train, X_val, y_val, fold_idx=None):
    C = trial.suggest_loguniform('logreg_C', 1e-3, 10)
    classes = np.unique(y_train)
    class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
    cw_dict = dict(zip(classes, class_weights))
    model = LogisticRegression(
        penalty='l2', C=C, solver='lbfgs', max_iter=1000,
        multi_class='multinomial', n_jobs=-1, random_state=42,
        class_weight=cw_dict
    )
    model.fit(X_train, y_train)
    preds_proba = model.predict_proba(X_val)
    f1 = f1_score(y_val, np.argmax(preds_proba, axis=1), average='weighted')

    if fold_idx is not None:
        save_path = os.path.join(OUTPUT_DIR, f"logreg_fold{fold_idx}.joblib")
        joblib.dump(model, save_path)
    return model, preds_proba, f1

# LightGBM
def train_lgb_optuna(trial, X_train, y_train, X_val, y_val, fold_idx=None):
    learning_rate = trial.suggest_loguniform('lgb_lr', 1e-3, 0.1)
    num_leaves = trial.suggest_int('lgb_num_leaves', 20, 50)
    max_depth = trial.suggest_int('lgb_max_depth', 3, 10)

    classes = np.unique(y_train)
    class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
    sample_weights = np.array([class_weights[np.where(classes == y)[0][0]] for y in y_train])

    model = lgb.LGBMClassifier(
        objective="multiclass",
        num_class=len(class_names),
        metric="multi_logloss",
        learning_rate=learning_rate,
        num_leaves=num_leaves,
        max_depth=max_depth,
        n_jobs=12,  # utilise 12 coeurs CPU
        random_state=42,
    )
    model.fit(X_train, y_train, sample_weight=sample_weights, eval_set=[(X_val, y_val)])
    preds_proba = model.predict_proba(X_val)
    f1 = f1_score(y_val, np.argmax(preds_proba, axis=1), average='weighted')

    if fold_idx is not None:
        save_path = os.path.join(OUTPUT_DIR, f"lgbm_fold{fold_idx}.joblib")
        joblib.dump(model, save_path)

    return model, preds_proba, f1

# MLP
class MetaMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, class_weights_tensor):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, n_classes)
        )
        self.criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

    def forward(self, x):
        return self.net(x)

def train_mlp_optuna(trial, X_train, y_train, X_val, y_val, fold_idx=None, epochs=20, batch_size=256):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    hidden_dim = trial.suggest_int('mlp_hidden_dim', 256, 1024)
    lr = trial.suggest_loguniform('mlp_lr', 1e-4, 1e-2)
    weight_decay = trial.suggest_loguniform('mlp_wd', 1e-6, 1e-3)

    classes = np.unique(y_train)
    class_weights_np = compute_class_weight('balanced', classes=classes, y=y_train)
    class_weights_tensor = torch.zeros(len(class_names), dtype=torch.float32)
    for c, w in zip(classes, class_weights_np):
        class_weights_tensor[c] = w
    class_weights_tensor = class_weights_tensor.to(device)

    model = MetaMLP(X_train.shape[1], hidden_dim, len(class_names), class_weights_tensor).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32).to(device), torch.tensor(y_train, dtype=torch.long).to(device))
    val_ds = TensorDataset(torch.tensor(X_val, dtype=torch.float32).to(device), torch.tensor(y_val, dtype=torch.long).to(device))
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size)

    best_f1 = 0
    best_state = None
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            logits = model(xb)
            loss = model.criterion(logits, yb)
            loss.backward()
            optimizer.step()

        model.eval()
        all_preds, all_targets = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                preds = torch.argmax(logits, dim=1)
                all_preds.append(preds.cpu().numpy())
                all_targets.append(yb.cpu().numpy())
        all_preds = np.concatenate(all_preds)
        all_targets = np.concatenate(all_targets)
        f1 = f1_score(all_targets, all_preds, average='weighted')
        if f1 > best_f1:
            best_f1 = f1
            best_state = model.state_dict()

    model.load_state_dict(best_state)
    model.eval()
    with torch.no_grad():
        val_probs = model(torch.tensor(X_val, dtype=torch.float32).to(device)).cpu().numpy()

    if fold_idx is not None:
        torch.save(best_state, os.path.join(OUTPUT_DIR, f"mlp_fold{fold_idx}.pth"))

    return model, val_probs, best_f1

# Threshold tuning
def threshold_tuning(y_true, probas, n_classes, initial_thresholds=None):
    thresholds = initial_thresholds if initial_thresholds is not None else np.ones(n_classes) * 0.5
    best_thresholds = thresholds.copy()
    best_f1 = 0

    for c in range(n_classes):
        thresh_range = np.linspace(0.1, 0.9, 9)
        for thr in thresh_range:
            thresholds[c] = thr
            preds = apply_thresholds(probas, thresholds)
            f1 = f1_score(y_true, preds, average='weighted')
            if f1 > best_f1:
                best_f1 = f1
                best_thresholds = thresholds.copy()
        thresholds[c] = best_thresholds[c]
    return best_thresholds

def apply_thresholds(probas, thresholds):
    n_samples, n_classes = probas.shape
    preds = np.zeros(n_samples, dtype=int)
    for i in range(n_samples):
        probs = probas[i]
        passed = np.where(probs >= thresholds)[0]
        if len(passed) == 0:
            preds[i] = np.argmax(probs)
        else:
            preds[i] = passed[np.argmax(probs[passed])]
    return preds

# Validation croisée sur combinaison fixe
def cross_val_train_tune_threshold_fixedcombo(logits_train_list, y_train, fixed_weights, classifiers, best_combo,
                                              n_splits=5, n_trials_hyper=50, n_trials_fusion=50):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    cv_scores = []

    for fold_idx, (train_fold_idx, val_fold_idx) in enumerate(skf.split(np.zeros(len(y_train)), y_train)):
        print(f"\n=== Fold {fold_idx+1}/{n_splits} ===")

        X_train_fold = weighted_fusion([logits[train_fold_idx] for logits in logits_train_list], fixed_weights)
        y_train_fold = y_train[train_fold_idx]

        X_val_fold = weighted_fusion([logits[val_fold_idx] for logits in logits_train_list], fixed_weights)
        y_val_fold = y_train[val_fold_idx]

        probas_list = []
        models_fold = {}

        for clf_name in best_combo:
            study = optuna.create_study(direction="maximize")
            func = lambda trial: classifiers[clf_name](trial, X_train_fold, y_train_fold, X_val_fold, y_val_fold, fold_idx)[2]
            # Parallélisation pour LightGBM et LogReg uniquement
            n_jobs_optuna = 12 if clf_name != "mlp" else 1
            study.optimize(func, n_trials=n_trials_hyper, show_progress_bar=True, n_jobs=n_jobs_optuna)
            best_trial = study.best_trial

            model, val_proba, f1 = classifiers[clf_name](best_trial, X_train_fold, y_train_fold, X_val_fold, y_val_fold, fold_idx)
            probas_list.append(val_proba)
            models_fold[clf_name] = model
            print(f"Fold {fold_idx+1} - F1 weighted {clf_name}: {f1:.4f}")

        study_fusion = optuna.create_study(direction="minimize")
        study_fusion.optimize(lambda trial: objective_fusion(trial, probas_list, y_val_fold), n_trials=n_trials_fusion, show_progress_bar=True)
        best_weights = np.array([study_fusion.best_params[f'w{i}'] for i in range(len(probas_list))])
        best_weights /= best_weights.sum()

        fusion_probas = weighted_fusion(probas_list, best_weights)
        best_thresholds = threshold_tuning(y_val_fold, fusion_probas, len(class_names))
        preds = apply_thresholds(fusion_probas, best_thresholds)
        f1 = f1_score(y_val_fold, preds, average='weighted')
        cv_scores.append(f1)

        print(f"Fold {fold_idx+1} - F1 weighted fusion: {f1:.4f}")
        print(classification_report(y_val_fold, preds, target_names=class_names, digits=4))

        # Sauvegarde
        for clf_name, model in models_fold.items():
            if clf_name == "mlp":
                torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, f"{clf_name}_fold{fold_idx}.pth"))
            else:
                joblib.dump(model, os.path.join(OUTPUT_DIR, f"{clf_name}_fold{fold_idx}.joblib"))

        np.save(os.path.join(OUTPUT_DIR, f"best_weights_fold{fold_idx}.npy"), best_weights)
        np.save(os.path.join(OUTPUT_DIR, f"best_thresholds_fold{fold_idx}.npy"), best_thresholds)

    print(f"\n=== CV mean F1 weighted: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f} ===")
    return np.mean(cv_scores), np.std(cv_scores)

# Entraînement final sur tout train+val
def train_final_with_tuning(logits_train_list, y_train, logits_val_list, y_val, fixed_weights, classifiers, best_combo,
                            n_trials_hyper=100, n_trials_fusion=100):
    X_train = weighted_fusion(logits_train_list, fixed_weights)
    X_val = weighted_fusion(logits_val_list, fixed_weights)

    probas_list = []

    for clf_name in best_combo:
        study = optuna.create_study(direction="maximize")
        func = lambda trial: classifiers[clf_name](trial, X_train, y_train, X_val, y_val)[2]
        n_jobs_optuna = 1 if clf_name == "mlp" else 12
        study.optimize(func, n_trials=n_trials_hyper, show_progress_bar=True, n_jobs=n_jobs_optuna)
        best_trial = study.best_trial

        model, val_proba, f1 = classifiers[clf_name](best_trial, X_train, y_train, X_val, y_val)
        if clf_name == "mlp":
            torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, f"final_{clf_name}.pth"))
        else:
            joblib.dump(model, os.path.join(OUTPUT_DIR, f"final_{clf_name}.joblib"))
        probas_list.append(val_proba)
        print(f"Final train F1 {clf_name}: {f1:.4f}")

    study_fusion = optuna.create_study(direction="minimize")
    study_fusion.optimize(lambda trial: objective_fusion(trial, probas_list, y_val), n_trials=n_trials_fusion, show_progress_bar=True)
    best_weights = np.array([study_fusion.best_params[f'w{i}'] for i in range(len(probas_list))])
    best_weights /= best_weights.sum()

    fusion_probas = weighted_fusion(probas_list, best_weights)
    best_thresholds = threshold_tuning(y_val, fusion_probas, len(class_names))

    preds = apply_thresholds(fusion_probas, best_thresholds)
    final_f1 = f1_score(y_val, preds, average='weighted')

    print("\n=== Résultats finaux avec tuning et seuils ===")
    print(f"Combinaison utilisée : {best_combo}")
    print(f"F1-weighted final : {final_f1:.4f}")
    print(classification_report(y_val, preds, target_names=class_names, digits=4))

    np.save(os.path.join(OUTPUT_DIR, "final_best_weights.npy"), best_weights)
    np.save(os.path.join(OUTPUT_DIR, "final_best_thresholds.npy"), best_thresholds)

    return best_weights, best_thresholds, final_f1

# Définition des classifieurs
classifiers = {
    "lgbm": train_lgb_optuna,
    "mlp": train_mlp_optuna,
    "logreg": train_logreg_optuna
}

# --- Lancer validation croisée uniquement sur la meilleure combinaison ---
mean_f1, std_f1 = cross_val_train_tune_threshold_fixedcombo(
    logits_train_list, y_train, fixed_weights,
    classifiers,
    best_combo=["lgbm", "mlp", "logreg"],
    n_splits=5,
    n_trials_hyper=50,
    n_trials_fusion=50
)

print(f"Validation croisée terminée : F1 moyen {mean_f1:.4f} ± {std_f1:.4f}")

# --- Entraînement final ---
best_weights, best_thresholds, final_f1 = train_final_with_tuning(
    logits_train_list, y_train, logits_val_list, y_val, fixed_weights,
    classifiers,
    best_combo=["lgbm", "mlp", "logreg"],
    n_trials_hyper=100,
    n_trials_fusion=100
)


[I 2025-06-09 19:11:50,191] A new study created in memory with name: no-name-e4ab56f3-84bf-47a9-9506-e38e89754c18



=== Fold 1/5 ===


  0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-1-1328526d0f0b>:120: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lgb_lr', 1e-3, 0.1)
<ipython-input-1-1328526d0f0b>:120: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lgb_lr', 1e-3, 0.1)
<ipython-input-1-1328526d0f0b>:120: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lgb_lr', 1e-3, 0.1)
<ipython-input-1-1328526d0f0b>:120: FutureWarning: suggest_loguniform has been deprec

[I 2025-06-09 19:12:41,931] Trial 4 finished with value: 0.9955677218610415 and parameters: {'lgb_lr': 0.08796066015839739, 'lgb_num_leaves': 36, 'lgb_max_depth': 4}. Best is trial 4 with value: 0.9955677218610415.


...

[I 2025-06-09 21:48:34,112] Trial 90 finished with value: 0.9191022116666363 and parameters: {'logreg_C': 0.0064570754230390565}. Best is trial 26 with value: 0.9194746854553782.
[I 2025-06-09 21:48:34,123] Trial 93 finished with value: 0.919258294431167 and parameters: {'logreg_C': 0.0034302797931489008}. Best is trial 26 with value: 0.9194746854553782.
[I 2025-06-09 21:48:34,345] Trial 95 finished with value: 0.9191748434895891 and parameters: {'logreg_C': 0.0031791724884523372}. Best is trial 26 with value: 0.9194746854553782.
[I 2025-06-09 21:48:34,361] Trial 92 finished with value: 0.9191022116666363 and parameters: {'logreg_C': 0.0066551929650080265}. Best is trial 26 with value: 0.9194746854553782.
[I 2025-06-09 21:48:34,374] Trial 96 finished with value: 0.919258294431167 and parameters: {'logreg_C': 0.0033427108080425452}. Best is trial 26 with value: 0.9194746854553782.
[I 2025-06-09 21:48:34,410] Trial 97 finished with value: 0.919258294431167 and parameters: {'logreg_C': 0.

<ipython-input-1-1328526d0f0b>:100: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('logreg_C', 1e-3, 10)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
[I 2025-06-09 21:48:35,531] A new study created in memory with name: no-name-6f3d3207-770b-4ed8-a122-69303350710c


Final train F1 logreg: 0.9195


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-06-09 21:48:35,560] Trial 0 finished with value: 0.08177960387196181 and parameters: {'w0': 0.8998636871373789, 'w1': 0.9931525336207844, 'w2': 0.41199970622595317}. Best is trial 0 with value: 0.08177960387196181.
[I 2025-06-09 21:48:35,577] Trial 1 finished with value: 0.0818621146834364 and parameters: {'w0': 0.7298425632897689, 'w1': 0.5893818704369191, 'w2': 0.01310727874080786}. Best is trial 0 with value: 0.08177960387196181.
[I 2025-06-09 21:48:35,591] Trial 2 finished with value: 0.08668277970024785 and parameters: {'w0': 0.892634546704932, 'w1': 0.004268668945744514, 'w2': 0.7183045732129486}. Best is trial 0 with value: 0.08177960387196181.
[I 2025-06-09 21:48:35,613] Trial 3 finished with value: 0.08177896981009392 and parameters: {'w0': 0.5753120285234827, 'w1': 0.3697242315246293, 'w2': 0.3543957514082613}. Best is trial 3 with value: 0.08177896981009392.
[I 2025-06-09 21:48:35,628] Trial 4 finished with value: 0.08165901434705669 and parameters: {'w0': 0.48621690

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 27.7 MB/s eta 0:00:00
